## Agenda

* The rearrange operator
   * Transposition of axes
   * Composition and Decomposition of axes
   * Add or remove axes
* The reduce operator
* The repeat operator
* torch.einsum for matrix multiplication.
* Layers vs. operations to build models.

## The Rearrange Operator


In [1]:
import torch
from einops import rearrange

# Transpose the last and the second axes
x = torch.randn((2, 3, 4, 5))
x = rearrange(x, 'b t h d -> b h t d')
x.shape

/Users/josiahdavis/miniconda3/envs/perceiver/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:276: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


torch.Size([2, 4, 3, 5])

In [2]:
# Compose the h and d dimension into a new dimension
# i.e., Collapse the last two dimensions together

x = torch.randn((2, 3, 4, 5))
x = rearrange(x, 'b t h d -> b t (h d)')
x.shape

torch.Size([2, 3, 20])

In [3]:
# Composition with the ellipses operator
x = torch.randn((2, 224, 224, 64))
x = rearrange(x, 'b ... d -> b (...) d')
x.shape

torch.Size([2, 50176, 64])

In [4]:
# Decompose is the inverse process
# i.e., Split out a single axis into multiple new axes
x = torch.randn((2, 3, 20))
x = rearrange(x, 'b t (h d) -> b t h d', h = 4, d = 5)
x.shape

torch.Size([2, 3, 4, 5])

In [5]:
# Addition of axes
# i.e., like torch.unsqueeze

x = torch.randn((2, 3, 4))
x = rearrange(x, 'b t d -> 1 b t d')
x.shape

torch.Size([1, 2, 3, 4])

In [6]:
# Removal of axes
# i.e., like torch.squeeze

x = torch.randn((1, 2, 3, 4))
x = rearrange(x, '1 b t d -> b t d')
x.shape

torch.Size([2, 3, 4])

## The Reduce Operator

Love this one.

In [7]:
from einops import reduce

x = torch.randn((2, 3, 4))
x = reduce(x, 'b t d -> b t', 'mean') # also supports max, min, std, sum and prod
x.shape

torch.Size([2, 3])

## The Repeat Operator

In [8]:
from einops import repeat

# Repeat along a single dimension
x = torch.randn((2, 3))
x = repeat(x, 't d -> b t d', b = 5)
x.shape

torch.Size([5, 2, 3])

In [9]:
# Repeat along multiple dimensions
x = torch.randn((2, 3))
x = repeat(x, 't d -> b h t d', b = 5, h = 4)
x.shape

torch.Size([5, 4, 2, 3])

In [10]:
# Using the ellipses format to avoid specifying existing dimensions, only the number of repeats!
x = torch.randn((2, 3, 4, 5))
x = repeat(x, '... -> b ...', b = 10)
x.shape

torch.Size([10, 2, 3, 4, 5])

## `torch.einsum` for Matrix Multiplication

In [11]:
from torch import einsum

# Standard format with 2D
a = torch.rand((3, 4))
b = torch.rand((4, 5))

c = einsum('i j, j d -> i d', a, b)
c.shape

torch.Size([3, 5])

In [12]:
# Standard format with 3D
q = torch.rand((2, 3, 4))
kt = torch.rand((2, 4, 5))

sim = einsum('b i j, b j d -> b i d', q, kt)
sim.shape

torch.Size([2, 3, 5])

In [13]:
# But you don't even need to do the transposition of axis
# It knows what you needed and does it for you

q = torch.rand((2, 3, 4))
k = torch.rand((2, 5, 4))

sim = einsum('b i j, b d j -> b i d', q, k)
sim.shape

torch.Size([2, 3, 5])

In [14]:
# Just to sanity check
torch.allclose(q @ k.transpose(-1, -2), sim)

True

## Layers vs. Operations to Build Models

Sometimes it is more convenient to use layers not operations to build models. This can be a huge space-saver.

In [15]:
import torch.nn as nn
from einops.layers.torch import Reduce

model = nn.Sequential(
    Reduce('b n d -> b d', 'mean'),
    nn.Linear(12, 14),
    nn.ReLU(),
    nn.Linear(14, 10),
)

model(torch.randn((2, 3, 12))).shape

torch.Size([2, 10])